In [4]:
from PIL import Image, ImageFont
import numpy as np
import random
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from tqdm import tqdm

class HopfieldNetwork(object):      
    def train_weights(self, train_data):
        print("Start to train weights...")
        num_data = len(train_data)
        self.num_neuron = train_data[0].shape[0]
        
        # initialize weights
        W = np.zeros((self.num_neuron, self.num_neuron))
        rho = np.sum([np.sum(t) for t in train_data]) / (num_data*self.num_neuron)
        
        # Hebb rule
        for i in tqdm(range(num_data)):
            print(rho)
            print(train_data)
            t = train_data[i] - rho
            print(t)
            W += np.outer(t, t)
        
        # Make diagonal element of W into 0
        diagW = np.diag(np.diag(W))
        W = W - diagW
        W /= num_data
        
        self.W = W 
    
    def predict(self, data, num_iter=20, threshold=0, asyn=False):
        print("Start to predict...")
        self.num_iter = num_iter
        self.threshold = threshold
        self.asyn = asyn
        
        # Copy to avoid call by reference 
        copied_data = np.copy(data)
        
        # Define predict list
        predicted = []
        for i in tqdm(range(len(data))):
            predicted.append(self._run(copied_data[i]))
        return predicted
    
    def _run(self, init_s):
        if self.asyn==False:
            """
            Synchronous update
            """
            # Compute initial state energy
            s = init_s
            e = self.energy(s)
            
            # Iteration
            for i in range(self.num_iter):
                # Update s
                s = np.sign(self.W @ s - self.threshold)
                # Compute new state energy
                e_new = self.energy(s)
                
                # s is converged
                if e == e_new:
                    return s
                # Update energy
                e = e_new
            return s
        else:
            """
            Asynchronous update
            """
            # Compute initial state energy
            s = init_s
            e = self.energy(s)
            
            # Iteration
            for i in range(self.num_iter):
                for j in range(100):
                    # Select random neuron
                    idx = np.random.randint(0, self.num_neuron) 
                    # Update s
                    s[idx] = np.sign(self.W[idx].T @ s - self.threshold)
                
                # Compute new state energy
                e_new = self.energy(s)
                
                # s is converged
                if e == e_new:
                    return s
                # Update energy
                e = e_new
            return s
    
    
    def energy(self, s):
        return -0.5 * s @ self.W @ s + np.sum(s * self.threshold)

    def plot_weights(self):
        plt.figure(figsize=(6, 5))
        w_mat = plt.imshow(self.W, cmap=cm.coolwarm)
        plt.colorbar(w_mat)
        plt.title("Network Weights")
        plt.tight_layout()
        plt.savefig("weights.png")
        plt.show()


FontSizes = [16, 32, 64]
NoisePercent = [0.1, 0.3, 0.6]
'''
for FontSize in FontSizes:
    font = ImageFont.truetype("Tahoma.ttf", FontSize)
    for char in "ABCDEFGHIJ":
        im = Image.Image()._new(font.getmask(char))
        im.save("Train" + str(FontSize) + "/" + char + ".bmp")
'''
'''
for Noise in NoisePercent:
    for FontSize in FontSizes:
        for char in "ABCDEFGHIJ":
            im = Image.open("Train" + str(FontSize) + "/" + char + ".bmp")
            ImageArray = np.array(im.convert('L'))
            IndexArray = []
            for i in range(0, len(ImageArray)):
                for j in range(0, len(ImageArray[0])):
                    IndexArray.append((i, j))
            ImageLength = len(ImageArray)*len(ImageArray[0])
            NoiseBit = int(ImageLength * Noise)
            SelectedIndex = random.sample(range(len(IndexArray)-1), NoiseBit)
            for Index in SelectedIndex:
                ImageArray[IndexArray[Index][0]][IndexArray[Index][1]] = random.randint(0, 255)
            NoisyIm = Image.fromarray(ImageArray, 'L')
            NoisyIm.save("Test" + str(int(Noise*100)) + "%" + str(FontSize) + "/" + char + ".bmp")
'''
for Noise in NoisePercent:
    for FontSize in FontSizes:
        TrainData = []
        TestData = []
        for char in "ABCDEFGHIJ":
            TrainData.append(np.array(Image.open("Train" + str(FontSize) + "/" + char + ".bmp")))
            TestData.append(np.array(Image.open("Test" + str(int(Noise*100)) + "%" + str(FontSize) + "/" + char + ".bmp")))
        Model = HopfieldNetwork()
        Model.train_weights(TrainData)
        Predicted = Model.predict(TestData, threshold=0, asyn=False)
        print("Show prediction results...")
        plot(data, test, predicted)
        print("Show network weights matrix...")
        model.plot_weights()


Start to train weights...


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

753.3166666666667
[array([[  0,   0,   0,   0,  82, 255, 231,   2,   0,   0,   0,   0],
       [  0,   0,   0,   0, 166, 243, 255,  61,   0,   0,   0,   0],
       [  0,   0,   0,   7, 242, 125, 236, 144,   0,   0,   0,   0],
       [  0,   0,   0,  77, 255,  35, 156, 227,   1,   0,   0,   0],
       [  0,   0,   0, 161, 205,   0,  70, 255,  56,   0,   0,   0],
       [  0,   0,   5, 238, 120,   0,   4, 236, 139,   0,   0,   0],
       [  0,   0,  72, 255,  35,   0,   0, 155, 222,   0,   0,   0],
       [  0,   0, 156, 255, 255, 255, 255, 255, 255,  51,   0,   0],
       [  0,   3, 235, 105,   0,   0,   0,   0, 226, 134,   0,   0],
       [  0,  67, 255,  37,   0,   0,   0,   0, 159, 218,   0,   0],
       [  0, 151, 225,   0,   0,   0,   0,   0,  91, 255,  46,   0],
       [  2, 232, 157,   0,   0,   0,   0,   0,  23, 254, 130,   0]],
      dtype=uint8), array([[  0, 208, 255, 255, 254, 245, 199,  88,   0,   0],
       [  0, 208, 188,   0,   1,  25, 156, 255,  59,   0],
       [  0, 2

ValueError: operands could not be broadcast together with shapes (12,12) (144,144) (12,12) 